In [1]:
import numpy as np
import torch
import sys
from pathlib import Path
import importlib

import pandas as pd
import torchaudio

module_path = str(Path.cwd().parents[0] / "utils")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if module_path not in sys.path:
    sys.path.append(module_path)
module_path = str(Path.cwd().parents[0] / "network_models/w2v_emotion_model")
if module_path not in sys.path:
    sys.path.append(module_path)
module_path = str(Path.cwd().parents[0] / "network_models/soundstream_lstm")
if module_path not in sys.path:
    sys.path.append(module_path)

import utils.wav2Vec_utils as w2vU
import utils.audio_dataset_utils as ADU
import network_models.soundstream_lstm.LSTM_dataset as lds
import network_models.soundstream_lstm.CombinedEmoDataset_7_Emo as ced

2023-01-25 16:49:19.261738: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-25 16:49:19.789880: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-25 16:49:19.789930: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-01-25 16:49:19.789935: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
importlib.reload(w2vU)
importlib.reload(lds)
importlib.reload(ced)

#tess = lds.AudioEmotionTessDataset(directory="/home/ckwdani/Music/emotionDatasets/converted_mono/tess", device="cpu")
emo_dataset = ced.CombinedEmoDataSet_7_emos(directory_tess="/home/ckwdani/Music/emotionDatasets/converted_mono/tess",
                                             directory_cafe="/home/ckwdani/Music/emotionDatasets/converted_mono/cafe",
                                             directory_ravdess="/home/ckwdani/Music/emotionDatasets/converted_mono/RAVDESS Audio_Speech_Actors_01-24",
                                             directory_mesd="/home/ckwdani/Music/emotionDatasets/converted_mono/mesd",
                                             device="cpu")
processor, sr = w2vU.init_w2v2(num_labels=len(emo_dataset.label_list), label_list=emo_dataset.label_list)

In [3]:
# maxnum = 0
# maxindex = 0
# for i in range(0, len(emo_dataset) - 1):
#     lo = emo_dataset[i][0].cpu()
#     size = (processor(lo)["input_values"][0][0].shape)[0]
#     maxnum = max(maxnum, size)
#     if(maxnum == size):
#         maxindex = i

In [4]:
# f,s = torchaudio.load(emo_dataset.dataFrame["path"][4640])
# len(f[0])
# lo = emo_dataset[6000][0].cpu()
# len(lo[0])
# processor(lo, return_tensors="pt")
# print(processor(lo[0], return_tensors="pt"), lo[0])

In [5]:
import datasets

importlib.reload(lds)

loaded = emo_dataset[500][0].cpu()
samplerate = emo_dataset.samplerate

newSet = lds.AudioEmotionTessWav2VecDataset(emo_dataset, processor= processor, sampling_rate=sr)
#res = torchaudio.transforms.Resample(orig_freq=samplerate, new_freq=sr)(loaded)
#output = processor(res)
#processor.pad()

In [6]:
#newSet.getEmotionFromId(np.where(newSet[1][1] == 1))

In [14]:
import network_models.w2v_emotion_model.custom_model as cm
import network_models.soundstream_lstm.model_trainer as ct

model = cm.W2V_EmotionClassifierSevenEmos().to(device=device)

In [15]:
importlib.reload(ADU)
trainSet, evalSet = ADU.train_val_dataset(newSet, 0.1)

In [16]:
emo_dataset.label_list

array(['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise'],
      dtype=object)

In [17]:
importlib.reload(ct)
trainer = ct.ModelTrainer(model=model, need_reshape=False, train_dataset=trainSet, eval_dataset=evalSet, device=device, batch_size=8, save_model_every=500, num_epochs=5001, model_path="content/classifier/W2VClassifier/Nr1/")

In [19]:
import gc
gc.collect()

trainer.train()

Epoch 1
-------------------------------
loss: 1.998926  [    0/ 5482]
loss: 1.857116  [  800/ 5482]
loss: 1.924270  [ 1600/ 5482]
loss: 1.919028  [ 2400/ 5482]
loss: 1.889499  [ 3200/ 5482]
loss: 2.016627  [ 4000/ 5482]
loss: 1.966058  [ 4800/ 5482]
Test Error: 
 Accuracy: 18.9%, Avg loss: 1.926252 

Epoch 2
-------------------------------
loss: 1.866223  [    0/ 5482]
loss: 1.861341  [  800/ 5482]
loss: 1.689579  [ 1600/ 5482]
loss: 1.890960  [ 2400/ 5482]
loss: 1.929978  [ 3200/ 5482]
loss: 1.935299  [ 4000/ 5482]
loss: 1.930234  [ 4800/ 5482]
Test Error: 
 Accuracy: 21.0%, Avg loss: 1.912487 

Epoch 3
-------------------------------
loss: 1.960669  [    0/ 5482]
loss: 1.845215  [  800/ 5482]
loss: 1.740980  [ 1600/ 5482]
loss: 1.926191  [ 2400/ 5482]
loss: 1.930356  [ 3200/ 5482]
loss: 1.836878  [ 4000/ 5482]
loss: 1.797034  [ 4800/ 5482]
Test Error: 
 Accuracy: 22.5%, Avg loss: 1.897633 

Epoch 4
-------------------------------
loss: 1.658308  [    0/ 5482]
loss: 1.598726  [  800/ 

KeyboardInterrupt: 

# Evaluation

In [20]:
#torch.save(model.state_dict(), "content/classifier/W2VClassifier/Nr1/model_3363.pth")

In [2]:
from transformers import AutoConfig, Wav2Vec2Processor, Wav2Vec2Config, Wav2Vec2Model
import network_models.soundstream_lstm.LSTM_dataset as lds
model_name_or_path = "lighteternal/wav2vec2-large-xlsr-53-greek"
pooling_mode = "mean"

tess = lds.AudioEmotionTessDataset(directory="/home/ckwdani/Music/emotionDatasets/converted_mono/tess")
# config
config = AutoConfig.from_pretrained(
    model_name_or_path,
    num_labels=len(tess.label_list),
    label2id={label: i for i, label in enumerate(tess.label_list)},
    id2label={i: label for i, label in enumerate(tess.label_list)},
    finetuning_task="wav2vec2_clf",
)
setattr(config, 'pooling_mode', pooling_mode)

processor = Wav2Vec2Processor.from_pretrained(model_name_or_path,)
target_sampling_rate = processor.feature_extractor.sampling_rate

conf = Wav2Vec2Config(name_or_path=model_name_or_path)
model = Wav2Vec2Model(conf)

2023-01-26 12:21:43 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX


Downloading:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/venv/lib/python3.10/site-packages/transformers/configuration_utils.py:369: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


Downloading:   0%|          | 0.00/158 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/138 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/535 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

In [30]:
import torch
torch.mean(model(tess[1000][0].cpu())[0], dim=1).shape

torch.Size([1, 768])